In [1]:
from guidance import models

# For LlamaCpp, you need to provide the path on disk to a .gguf model
# A sample model can be downloaded from
# https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/blob/main/mistral-7b-instruct-v0.2.Q8_0.gguf
#mistral = models.LlamaCpp("/home/scottlundberg_google_com/models/mistral-7b-instruct-v0.2.Q8_0.gguf", n_gpu_layers=-1, n_ctx=4096)

#llama2 = models.Transformers("meta-llama/Llama-2-7b-hf")
#gpt = models.OpenAI("gpt-3.5-turbo")
gpt = models.OpenAI("text-davinci-003")
#palm2 = models.VertexAI("text-bison@001")

In [2]:
lm = gpt + "Who won the last Kentucky derby and by how much?"

In [5]:
from guidance import gen, user, assistant

gpt = models.OpenAI("gpt-3.5-turbo")

with user():
    lm = gpt + "What is the capital of France?"

with assistant():
    lm += gen("capital")

with user():
    lm += "What is one short surprising fact about it?"

with assistant():
    lm += gen("fact")

In [7]:
from guidance import select, guidance, models, gen, user, assistant, system

# Define the guidance model using OpenAI interface
gpt = models.OpenAI("gpt-3.5-turbo")

@guidance
def qa_bot(lm, query):
    with user():
        lm += f'''Q: {query}
        Now I will choose to either SEARCH the web or RESPOND.
        Choice: {select(["SEARCH", "RESPOND"], name="choice")}'''

    if lm["choice"] == "SEARCH":
        with assistant():
            lm += "A: I don't know, Google it!"
    else:
        with assistant():
            lm += f'A: {gen(stop="Q:", name="answer")}'
    return lm

query = "Who won the last Kentucky derby and by how much?"

# Run the qa_bot function within the assistant context
with assistant():
    lm = gpt + qa_bot(query)


TypeError: qa_bot() takes 2 positional arguments but 3 were given